In [43]:
import os
import pyspark
from pyspark.sql.types import StructType,StructField, StringType, IntegerType 
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
datasets_path = os.path.join('..','movie_recommendation_system', 'datasets')
complete_ratings_file = os.path.join(datasets_path, 'ml-latest', 'ratings.csv')
tags_file = os.path.join(datasets_path, 'ml-latest', 'tags.csv')
links_file = os.path.join(datasets_path, 'ml-latest','links.csv')
movies_file = os.path.join(datasets_path, 'ml-latest', 'movies.csv')

In [44]:
spark = SparkSession.builder.getOrCreate()

In [45]:
#Schema design which are as same as csv
ratingschema = StructType()\
    .add("userID", IntegerType(), True)\
    .add("movieId", IntegerType(), True)\
    .add("rating", IntegerType(), True)\
    .add("timeStamp", IntegerType(), True)
tagsschema = StructType()\
    .add("userID", IntegerType(), True)\
    .add("movieId", IntegerType(), True)\
    .add("tag", StringType(), True)\
    .add("timeStamp", IntegerType(), True)
linkschema = StructType()\
    .add("movieId", IntegerType(), True)\
    .add("IMDBID", IntegerType(), True)\
    .add("TMDBID", IntegerType(), True)

In [46]:
ratingdf = spark.read.format("csv")\
    .option("header",True)\
    .schema(ratingschema)\
    .load(complete_ratings_file)
tagdf = spark.read.format("csv")\
    .option("header",True)\
    .schema(tagsschema)\
    .load(tags_file)
linkdf = spark.read.format("csv")\
    .option("header",True)\
    .schema(linkschema)\
    .load(links_file)

In [48]:
movie = spark.read.option("inferSchema", "true")\
    .option("header", "true").csv(movies_file)
moviedf = moviedf.drop("genres")
genredf = movie.drop("title")
userdf = ratingdf.drop("movieID", "tating", "timeStamp")
userdf = userdf.select('userID').distinct()